# Obiettivo del Progetto

L’obiettivo di questo progetto è valutare le prestazioni di PostgreSQL eseguendo alcune query di selezione, proiezione e join in due configurazioni differenti:

1. **Senza indici**: oltre alle chiavi primarie di base.
2. **Con indici**: creati ad hoc su determinati campi.

Si confronteranno i piani di esecuzione (mediante `EXPLAIN ANALYZE`) e si misureranno, opzionalmente, i blocchi letti/hittati (pin e read) tramite la vista di sistema `pg_stat_database`.

---
### Generazione di Dati Fittizi con Faker

In questa sezione, utilizzeremo la libreria Python `faker` per generare dati fittizi. Questo può essere utile per simulare un database o per testare query SQL. Lo script seguente crea un file CSV con dati separati da un delimitatore personalizzato (ad esempio, `|`).


In [5]:
import psycopg2
from faker import Faker
import random
import subprocess
import os

In [17]:
import os
import subprocess

print("Current working directory:", os.getcwd())

def run_sql_script(script_path, db_name, db_user, db_password, db_host="localhost"):
    env = os.environ.copy()
    env["PGPASSWORD"] = db_password

    cmd = [
        "psql",
        "-U", db_user,
        "-d", db_name,
        "-h", db_host,
        "-f", script_path
    ]
    
    print(f"Esecuzione di {script_path}...")
    subprocess.run(cmd, check=True, env=env)
    print(f"{script_path} completato.\n")

if __name__ == "__main__":
    # Parametri di connessione
    DB_NAME = "universita"      # Sostituisci con il nome del tuo database
    DB_USER = "postgres"      # Sostituisci se necessario
    DB_PASSWORD = "postgres"  # Sostituisci con la tua password
    DB_HOST = "localhost"
    
    # Percorsi dei file SQL (in questo esempio, nella directory corrente)
    create_tables_path = os.path.join(os.getcwd(), "create_tables.sql")
    create_indexes_btree_path = os.path.join(os.getcwd(), "create_indexes_btree.sql")
    create_indexes_hash_path = os.path.join(os.getcwd(), "create_indexes_hash.sql")

Current working directory: c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2


### Configurazione A senza indici

In [ ]:
# Per configurazione A (senza indice)
print("Configurazione A: Senza indice")
run_sql_script(create_tables_path, DB_NAME, DB_USER, DB_PASSWORD, DB_HOST)

Configurazione A: Senza indice
Esecuzione di c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_tables.sql...
c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_tables.sql completato.



### Configurazione B (Indice B-Tree su id_corso)

In [25]:
# Per configurazione B (con indice B-Tree)
print("Configurazione B: Con indice B-Tree")
run_sql_script(create_indexes_btree_path, DB_NAME, DB_USER, DB_PASSWORD, DB_HOST)

Configurazione B: Con indice B-Tree
Esecuzione di c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_indexes_btree.sql...
c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_indexes_btree.sql completato.



### Configurazione C (Indice Hash su corso)

In [9]:
# poi esegui l'indice Hash:
print("Configurazione C: Con indice Hash")
run_sql_script(create_indexes_hash_path, DB_NAME, DB_USER, DB_PASSWORD, DB_HOST)

Configurazione C: Con indice Hash
Esecuzione di c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_indexes_hash.sql...
c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_indexes_hash.sql completato.



In [7]:

# Parametri di connessione al database
DB_NAME = "universita"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"  # o l'host appropriato

fake = Faker()

# Dimensioni dei dati
N_STUDENTI = 50000
N_CORSI = 5000
N_ISCRIZIONI = 2000000

# Connessione al database
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
cur = conn.cursor()

# Svuota le tabelle (se esistono dati)
cur.execute("TRUNCATE TABLE Iscrizione CASCADE;")
cur.execute("TRUNCATE TABLE Corso CASCADE;")
cur.execute("TRUNCATE TABLE Studente CASCADE;")

# Popola la tabella Studente
for i in range(1, N_STUDENTI + 1):
    nome = fake.first_name()
    email = fake.email()
    indirizzo = fake.address().replace("\n", ", ")
    cur.execute(
        "INSERT INTO Studente (id, nome, email, indirizzo) VALUES (%s, %s, %s, %s);",
        (i, nome, email, indirizzo)
    )

# Popola la tabella Corso
for i in range(1, N_CORSI + 1):
    nome_corso = "Corso_" + fake.word()
    cfu = random.randint(3, 12)
    cur.execute(
        "INSERT INTO Corso (id, nome, cfu) VALUES (%s, %s, %s);",
        (i, nome_corso, cfu)
    )

# Popola la tabella Iscrizione
for _ in range(N_ISCRIZIONI):
    stud_id = random.randint(1, N_STUDENTI)
    corso_id = random.randint(1, N_CORSI)
    voto = random.randint(18, 30)
    cur.execute(
        "INSERT INTO Iscrizione (studente_id, corso_id, voto) VALUES (%s, %s, %s);",
        (stud_id, corso_id, voto)
    )

conn.commit()
cur.close()
conn.close()

print("Popolamento completato!")


Popolamento completato!


## Visualizzazione dati nelle relazioni

In [8]:
import psycopg2

# Connessione al database
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
cur = conn.cursor()

# Relazioni da visualizzare
relazioni = ["Studente", "Corso", "Iscrizione"]

for relazione in relazioni:
    print(f"Primi 10 record della tabella {relazione}:")
    cur.execute(f"SELECT * FROM {relazione} LIMIT 10;")
    records = cur.fetchall()
    for record in records:
        print(record)
    print("\n")

cur.close()
conn.close()


Primi 10 record della tabella Studente:
(1, 'Eric', 'jennifer09@example.org', '58491 Michelle Grove, Port Caitlin, KS 74463')
(2, 'Corey', 'tcain@example.org', 'USNS Chen, FPO AE 56329')
(3, 'David', 'awhite@example.org', '1625 Tina Square Suite 656, East Amandaville, NM 41694')
(4, 'Ian', 'tyler75@example.net', '42229 Monica Manors Suite 688, Tuckerhaven, NH 36034')
(5, 'Rhonda', 'timothy03@example.org', '44274 Hernandez Avenue Apt. 319, South Amyhaven, TN 20797')
(6, 'Jacob', 'wallanna@example.net', 'Unit 5516 Box 6079, DPO AA 16022')
(7, 'Jaime', 'xjacobs@example.org', '348 Fields Trail, East Ritastad, VA 87101')
(8, 'Keith', 'christina98@example.com', '016 Patricia Ports, East Stanley, NC 13408')
(9, 'Michael', 'lindabrown@example.com', '886 Matthew Stream, New Martha, CA 40359')
(10, 'Michelle', 'michael55@example.net', '692 Christopher Trail, Dustinstad, NJ 09047')


Primi 10 record della tabella Corso:
(1, 'Corso_tree', 10)
(2, 'Corso_lose', 10)
(3, 'Corso_company', 10)
(4, 'Cor

# Parte 2 del Progetto

La seconda parte del progetto consiste nel definire ed eseguire le query di test per confrontare i piani di esecuzione e le prestazioni nelle diverse configurazioni (senza indice, con indice B-Tree e con indice Hash).

Questa funzione esegue i seguenti passaggi:

1. **Drop della tabella esistente**: Se esiste una tabella con lo stesso nome, viene eliminata.
2. **Creazione di una nuova tabella**: La tabella viene creata con i valori iniziali di `blks_hit + blks_read` e `blks_read`, letti dalla vista `pg_stat_database`.
3. **Esecuzione della query da testare**: La query viene eseguita per misurare le prestazioni.
4. **Attesa opzionale**: È possibile inserire un'attesa per stabilizzare le statistiche.
5. **Lettura dei valori finali**: I valori finali di `blks_hit` e `blks_read` vengono letti nuovamente da `pg_stat_database`.
6. **Calcolo delle differenze**: Viene calcolata la differenza tra i valori iniziali e finali per ottenere i risultati di `pin` (numPin) e `read` (numRead).
7. **Stampa dei risultati**: I valori risultanti vengono stampati a schermo.

**Nota importante**: Nel comando `WHERE datname LIKE 'postgres'`, sostituisci `'postgres'` con il nome del database che stai utilizzando (ad esempio, `'universita'` se il tuo database si chiama “universita”).


In [ ]:
import psycopg2
import time

# Parametri di connessione
DB_NAME = "universita"   # o "postgres", se il DB si chiama così
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"

queries = [
    {
        "name": "Query 1: join + selezione",
        "sql": """
            EXPLAIN ANALYZE
            SELECT s.nome, s.email, i.voto
            FROM Studente s
            JOIN Iscrizione i ON s.id = i.studente_id
            WHERE i.corso_id=10;
        """
    },
    {
        "name": "Query 2: range scan su corso_id",
        "sql": """
            EXPLAIN ANALYZE
            SELECT s.nome, s.email, i.voto
            FROM Studente s
            JOIN Iscrizione i ON s.id = i.studente_id
            WHERE i.corso_id BETWEEN 100 AND 200
            ORDER BY i.corso_id;
        """
    }
]

def get_stats(cur):
    """
    Recupera le statistiche correnti da pg_stat_database per il database DB_NAME.
    Ritorna una tupla: (numPin, numRead) dove:
      - numPin = blks_hit + blks_read (accessi totali ai blocchi)
      - numRead = blks_read (accessi fisici)
    """
    cur.execute("""
        SELECT blks_hit + blks_read, blks_read
        FROM pg_stat_database
        WHERE datname = %s;
    """, (DB_NAME,))
    return cur.fetchone()

def run_explain_analyze(connection, query_dict):
    """
    Esegue la EXPLAIN ANALYZE e ritorna l'output (lista di righe).
    """
    cur = connection.cursor()
    cur.execute(query_dict["sql"])
    rows = cur.fetchall()  # Ogni riga è una tupla con una singola colonna di testo
    cur.close()
    return rows

def run_query_with_stats(connection, query_dict):
    """
    Per la query indicata:
      - Recupera le statistiche I/O prima dell'esecuzione.
      - Esegue EXPLAIN ANALYZE e raccoglie il piano di esecuzione.
      - Attende un attimo per aggiornare le statistiche.
      - Recupera le statistiche I/O dopo l'esecuzione.
      - Calcola le differenze (differenza di numPin e numRead).
    Ritorna un dizionario con:
      - "name": nome della query
      - "plan": lista di righe del piano di esecuzione
      - "numPin_diff": differenza dei blocchi (hit+read)
      - "numRead_diff": differenza dei blocchi letti fisicamente
    """
    cur = connection.cursor()
    
    # Statistiche prima della query
    stats_before = get_stats(cur)
    
    # Esecuzione della query (EXPLAIN ANALYZE)
    plan_output = run_explain_analyze(connection, query_dict)
    
    connection.commit()
    time.sleep(2)  # Attesa per permettere l'aggiornamento delle statistiche
    
    # Statistiche dopo la query
    stats_after = get_stats(cur)
    cur.close()
    
    # Calcolo delle differenze
    numPin_diff = stats_after[0] - stats_before[0]
    numRead_diff = stats_after[1] - stats_before[1]
    
    return {
        "name": query_dict["name"],
        "plan": plan_output,
        "numPin_diff": numPin_diff,
        "numRead_diff": numRead_diff
    }

# Configurazione attuale: modifica a mano in base alla configurazione impostata ("NO_INDEX", "BTREE", "HASH")
config = "NO_INDEX"

# Apertura della connessione
# Apertura della connessione
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
conn.autocommit = True

# Disabilita gli index scan per questa sessione
cur = conn.cursor()
#cur.execute("SET enable_indexscan = off;")
#cur.execute("SET enable_bitmapscan = off;")
cur.close()

print(f"==> Test PRE-ANALYZE ({config})")
results_pre = []
for q in queries:
    print(q["name"])
    result = run_query_with_stats(conn, q)
    # Stampa il piano di esecuzione
    print("Piano di esecuzione:")
    for line in result["plan"]:
        print(line[0])
    # Stampa le statistiche I/O
    print(f"  --> Differenza Num Pin: {result['numPin_diff']}, Differenza Num Read: {result['numRead_diff']}")
    print("-" * 60)
    results_pre.append(result)

print("==> Ora eseguo ANALYZE globale...")
cur = conn.cursor()
cur.execute("ANALYZE;")
cur.close()

print(f"==> Test POST-ANALYZE ({config})")
results_post = []
for q in queries:
    print(q["name"])
    result = run_query_with_stats(conn, q)
    print("Piano di esecuzione:")
    for line in result["plan"]:
        print(line[0])
    print(f"  --> Differenza Num Pin: {result['numPin_diff']}, Differenza Num Read: {result['numRead_diff']}")
    print("-" * 60)
    results_post.append(result)

conn.close()

# A questo punto, i dizionari 'results_pre' e 'results_post' contengono tutte le informazioni
# (piani di esecuzione e differenze nelle statistiche I/O) utili per l'analisi e la relazione.


==> Test PRE-ANALYZE (NO_INDEX)
Query 1: join + selezione
Piano di esecuzione:
Hash Join  (cost=1816.51..3163.46 rows=398 width=33) (actual time=15.240..17.481 rows=388 loops=1)
  Hash Cond: (i.studente_id = s.id)
  ->  Bitmap Heap Scan on iscrizione i  (cost=7.51..1353.41 rows=398 width=8) (actual time=0.371..2.445 rows=388 loops=1)
        Recheck Cond: (corso_id = 10)
        Heap Blocks: exact=384
        ->  Bitmap Index Scan on idx_iscrizione_corso_btree  (cost=0.00..7.41 rows=398 width=0) (actual time=0.339..0.339 rows=388 loops=1)
              Index Cond: (corso_id = 10)
  ->  Hash  (cost=1184.00..1184.00 rows=50000 width=33) (actual time=14.739..14.740 rows=50000 loops=1)
        Buckets: 65536  Batches: 1  Memory Usage: 3801kB
        ->  Seq Scan on studente s  (cost=0.00..1184.00 rows=50000 width=33) (actual time=0.025..7.562 rows=50000 loops=1)
Planning Time: 2.873 ms
Execution Time: 18.189 ms
  --> Differenza Num Pin: 0, Differenza Num Read: 0
---------------------------